# Pipeline to run all files
* All scripts currently output Excel files because we were working independently
* <b>We'll be changing this so that each script passes dataframes until the end of the pipeline.</b>
* Our plan is to not store anything (refresh daily) but need to consider how to deal with taking in user feedback

In [6]:
#Gather Articles
import NewsAPI as news
import os

#delete any old sorted article files
#if os.path.exists("articles.csv"):
#    os.remove("articles.csv")

#inputs in order: set manual date (yes=1/no=0, no = last 24 hours of news), manual start date (leave empty if 0), manual end date (leave empty if 0, companies to pull (6= all 19 companies)
articleDB = news.main(0,"","",6) #output is called 'NewsAPIOutput.xlsx' in Python Scripts > Data folder


Gathering articles on (Gap Inc) OR (Foot Locker) OR (L Brands) OR Macerich OR Kimco OR TJX OR CVS OR (Home Depot) OR (Best Buy) OR (Lowe's) OR Walmart OR (Target's) OR TGT OR Amazon OR Kroger OR Walgreens OR Kohl's OR (Dollar General) OR (Bed Bath and Beyond) OR Safeway from: 2019-02-11T23:12:00 to 2019-02-12T23:12:00
158


In [7]:
#Article Cleaning (must pip install tqdm first (only once) to run)
import dataClean as dc
cleanedArticles = dc.DataClean(articleDB)

In [8]:
#Feature Selection and Binary Article Encoding
import FeatureEncoding as fe
binaryMatrix = fe.encoding(0, df=articleDB)

Binary Encoding


100%|██████████████████████████████████████████████████████████████████████████████| 158/158 [00:00<00:00, 3960.63it/s]


In [9]:
#Logistic Regression Classifier + Article Ranking, complete final file is called 'results_encoding.xlsx'
import logReg as lr
articleDB = lr.runLogReg(binaryMatrix, articleDB)

C:\Users\Legendary\Anaconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


                                               title  \
0  Omar smashed over sneers at Jewish money in po...   
1     Can the middle-class revival under Trump last?   
2  Excerpts from recent editorials in newspapers ...   
3  30 acres once owned by Abraham Lincoln in Illi...   
4  Fyre Festival to fashion week, how Instagram i...   

                                         description  
0  Omar smashed over sneers at Jewish money in po...  
1  In his State of the Union, Trump said the midd...  
2                                  February 10, 2019  
3  30 acres once owned by Abraham Lincoln in Illi...  
4  YouTube and Instagram influencers make hundred...  


In [10]:
# 1) Extract key phrases, unigrams, and uni+bigrams
# complete final file is called 'results_context.xlsx'
# first input is filename 
# (optional) second input is tag type (could be 'ngrams'{unlimited}, 'bigrams'{terms with up to 2 words}, or 'unigrams'{single terms})
#    future work: add in noun phrases, named entities
# 2) outputs "trending_terms.xlsx" which outputs keyterms by # article mentions
import ContextExtraction as ce
articleDB = ce.retrieveContext(articleDB)

AttributeError: 'NoneType' object has no attribute 'type'

In [ ]:
articleDB.head()

In [6]:
import Recommender as rec
FinalTable = rec.recommender("binEncoding.csv", "results_context.xlsx")


0
[ 88  32 102   0]
[ 88  32 102   0]
[0.38177086 0.41602515 0.49613894 1.        ]
1
[ 49 132  76   1]
[ 49 132  76   1]
[0.28867513 0.35355339 0.37796447 1.        ]
2
[133  15   2  96]
[133  15  96   2]
[0.3928371  0.42163702 0.4330127  1.        ]
3
[ 49  47  46 143]
[ 49  47  46 143]
[0. 0. 0. 0.]
4
[113   4  84 102]
[113  84 102   4]
[0.33333333 0.4472136  0.51639778 1.        ]
5
[ 69 110  96   5]
[ 69 110  96   5]
[0.38490018 0.40824829 0.41247896 1.        ]
6
[ 99 129  27   6]
[ 99  27 129   6]
[0.35355339 0.35355339 0.42640143 1.        ]
7
[24 23 41  7]
[24 23 41  7]
[0.4472136  0.4472136  0.57735027 1.        ]
8
[13 35 25  8]
[13 25 35  8]
[0.23904572 0.23904572 0.26726124 1.        ]
9
[  7  24 137   9]
[  7  24 137   9]
[0.31139958 0.34815531 0.34815531 1.        ]
10
[28 26 43 10]
[28 26 43 10]
[0.47140452 0.47140452 0.50395263 1.        ]
11
[135  74 102  11]
[135  74 102  11]
[0.36514837 0.39036003 0.4        1.        ]
12
[109 115  76  12]
[109  76 115  12]
[0.3779

[124   0   4 102]
[0.4472136  0.49613894 0.51639778 1.        ]
103
[ 84  79 103  73]
[ 84  79  73 103]
[0.37573457 0.38348249 0.41251432 1.        ]
104
[ 28 127  26 104]
[ 28 127  26 104]
[0.35355339 0.35355339 0.35355339 1.        ]
105
[ 49  47  46 143]
[ 49  47  46 143]
[0. 0. 0. 0.]
106
[ 37 141  98 106]
[ 37 141  98 106]
[0.42163702 0.46225016 0.47140452 1.        ]
107
[ 50  59  16 107]
[ 50  59  16 107]
[0.40824829 0.40824829 0.40824829 1.        ]
108
[  6  17 108  93]
[  6  17  93 108]
[0.35355339 0.37796447 0.41602515 1.        ]
109
[115  12  84 109]
[115  12  84 109]
[0.3086067  0.37796447 0.39036003 1.        ]
110
[  5  92 110  94]
[  5  94  92 110]
[0.40824829 0.4330127  0.47140452 1.        ]
111
[ 97  79  48 111]
[ 97  79  48 111]
[0.28867513 0.31622777 0.37796447 1.        ]
112
[ 88 132  46 112]
[ 88 132  46 112]
[0.36842105 0.40555355 0.41502868 1.        ]
113
[ 32  84   4 113]
[ 32  84   4 113]
[0.28867513 0.2981424  0.33333333 1.        ]
114
[  6  20 114 123]


In [7]:
FinalTable.head()

,nonRel,Rel,article_id,prediction,difference,title,description,url,source,date,content,origContent,tags,tags_top_5,related_articles
0,-1.696713,-0.202465,93,1,1.494248,Omar smashed over sneers at Jewish money in po...,Omar smashed over sneers at Jewish money in po...,https://www.foxnews.com/politics/omar-smashed-...,fox-news,2019-02-11 23:26:56,Want FOX News Halftime Report in your inbox ev...,**Want FOX News Halftime Report in your inbox ...,"want fox, halftime, congress, democratic leade...","hope,personal biography,horizontal reading,due...","146, 153, 73"
1,-1.365472,-0.294720,150,1,1.070751,Excerpts from recent editorials in newspapers ...,"February 10, 2019",https://apnews.com/1a0371c3dbeb437284d57c3c982...,associated-press,2019-02-12 18:09:15,February 10 2019Chicago TribuneMeasles surges ...,"February 10, 2019\r\nChicago Tribune\r\nMeasle...","tribunemeasles, even death, public demand, her...","forms remuneration,states fatalities,initial r...","146, 134, 128"
2,-1.215559,-0.351750,106,1,0.863809,Cramer: This earnings period revealed 'brutal ...,Jim Cramer guides investors on how to approach...,https://www.cnbc.com/2019/02/11/cramer-earning...,cnbc,2019-02-11 23:11:00,If the latest round of earnings reports taught...,If the latest round of earnings reports taught...,"traders arent, uschina, nike, starbucks, ameri...","pain techs,peoples,unassailable brands,applequ...","44, 117, 103"
3,-0.796482,-0.599497,9,1,0.196985,How to properly clean an iPad screen without d...,You'll want to avoid using sprays or any semi-...,https://www.businessinsider.com/how-to-clean-i...,business-insider,2019-02-11 21:21:00,My device screens are perpetually collecting g...,My device screens are perpetually collecting g...,"device screens, maybe smog, screen ipad, stron...","perimeter ipad,cracks ipad,abrasive cloths,dir...","29, 60, 32"
4,-0.761865,-0.628849,134,1,0.133016,Can the middle-class revival under Trump last?,"In his State of the Union, Trump said the midd...",https://www.usatoday.com/story/money/2019/02/1...,usa-today,2019-02-12 11:01:05,In his first State of the Union address before...,In his first State of the Union address before...,"congress, washington, good country, clear path...","reeves,middle class,highwage jobs,mckinsey,mar...","20, 92, 140"
